In [41]:
import torch
import matplotlib.pyplot as plt
from torch import nn

In [42]:
torch.__version__

'2.5.1+cu121'

In [43]:
## Set upto the cuda device

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# Create a Linear Regresion formula   $y= m * x + b$

In [44]:
weight = 0.7
bias = 0.2

# create range values
start = 0
end = 10
step = 0.2


# Create X and Y (feature and label)
x = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * x  + bias

In [45]:
x[:10],y[:10]

(tensor([[0.0000],
         [0.2000],
         [0.4000],
         [0.6000],
         [0.8000],
         [1.0000],
         [1.2000],
         [1.4000],
         [1.6000],
         [1.8000]]),
 tensor([[0.2000],
         [0.3400],
         [0.4800],
         [0.6200],
         [0.7600],
         [0.9000],
         [1.0400],
         [1.1800],
         [1.3200],
         [1.4600]]))

In [46]:
train_split = int(0.8 * len(x))
X_train, y_train = x[:train_split], y[:train_split]
X_test, y_test = x[train_split:], y[train_split:]

In [47]:
len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)

### Building a Pytorch Linear Model

In [48]:
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.Linear_layer = nn.Linear(in_features =1, out_features= 1)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.Linear_layer(x)

In [49]:
# create a manual seed for over model prediction 
torch.manual_seed(42)

# then we are called over model
model = LinearRegression()
model, model.state_dict()

(LinearRegression(
   (Linear_layer): Linear(in_features=1, out_features=1, bias=True)
 ),
 OrderedDict([('Linear_layer.weight', tensor([[0.7645]])),
              ('Linear_layer.bias', tensor([0.8300]))]))

In [52]:
%%timeit
next(model.parameters()).device

4.84 μs ± 255 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [53]:
%%timeit
model.to(device)
next(model.parameters()).device

22.9 μs ± 1.21 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Training 
### For training Loop

* Loss function
* optimizer
* Training Loop
* Testing Loop
  

In [54]:
# setup loss function

loss_fn = nn.L1Loss() # same as MAE


In [55]:
# Setup our optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

## Let's Write a Training Loop

In [56]:

torch.manual_seed(42)
epochs = 200

for epoch in range(epochs):

    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)

    model.train()

    # 1. Forward pass
    y_pred = model(X_train)

    # 2.Loss function
    loss = loss_fn(y_pred, y_train)

    # 3.optim
    optimizer.zero_grad()

    # 4.perform backpropagation
    loss.backward()

    # 5.optimizer stpe
    optimizer.step()

    # Testing model

    model.eval()
    with torch.inference_mode():
        test_pred = model(X_test)

        test_loss = loss_fn(test_pred, y_test)

    ## Print out whatsHappening

    if epoch % 10 == 0:
        print(f"epoch : {epoch} | train loss{loss} | test loss {test_loss}")

epoch : 0 | train loss0.8817082643508911 | test loss 0.8473014831542969
epoch : 10 | train loss0.2374504804611206 | test loss 0.33865752816200256
epoch : 20 | train loss0.22027067840099335 | test loss 0.3226965367794037
epoch : 30 | train loss0.20297841727733612 | test loss 0.2813151776790619
epoch : 40 | train loss0.1858103722333908 | test loss 0.26535430550575256
epoch : 50 | train loss0.16864238679409027 | test loss 0.2493932694196701
epoch : 60 | train loss0.1513383388519287 | test loss 0.20801196992397308
epoch : 70 | train loss0.13418234884738922 | test loss 0.19294153153896332
epoch : 80 | train loss0.11717834323644638 | test loss 0.1533416360616684
epoch : 90 | train loss0.09974632412195206 | test loss 0.13827137649059296
epoch : 100 | train loss0.0827857032418251 | test loss 0.12409181892871857
epoch : 110 | train loss0.07601316273212433 | test loss 0.3162216246128082
epoch : 120 | train loss0.07348845154047012 | test loss 0.026519346982240677
epoch : 130 | train loss0.0406115

In [57]:
torch.save(model.state_dict(),'linear_model.pth')

In [59]:
model.state_dict()

OrderedDict([('Linear_layer.weight', tensor([[0.6606]], device='cuda:0')),
             ('Linear_layer.bias', tensor([0.2060], device='cuda:0'))])

## Load my model into the deployment

In [63]:
import warnings
warnings.filterwarnings('ignore')

model.load_state_dict(torch.load("Linear_model.pth"))

<All keys matched successfully>

In [64]:
model.eval()

LinearRegression(
  (Linear_layer): Linear(in_features=1, out_features=1, bias=True)
)